## Drive Loading...

In [1]:
from google.colab import drive
drive.mount("/content/drive")
% cd '/content/drive/MyDrive/Università/Models/Reinvent-master'
! ls
! nvidia-smi

Mounted at /content/drive
/content/drive/MyDrive/Università/Models/Reinvent-master
chembl_mcp_models  Miniconda3-py37_4.8.2-Linux-x86_64.sh  running_modes
cmp_model.py	   model_container.py			  scaffold
data		   models				  scoring
__init__.py	   __pycache__				  Training
input.py	   reinvent_shared.yml			  utils
main_test.py	   Results
Sat Jun 26 07:16:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |            

## Environment Setup

In [2]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
import os
import sys
if not os.path.exists('Miniconda3-py37_4.8.2-Linux-x86_64.sh'):
    ! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
sys.path.append('/usr/local/lib/python3.7/site-packages/')

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b6447c_0
    - conda==4.8.2=py37_0
    - cryptography==2.8=py37h1ba5d50_0
    - idna==2.8=py37_0
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20181209=hc058e9b_0
    - libffi==3.2.1=hd88cf55_4
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_0
    - openssl==1.1.1d=h7b6447c_4
    - pip==20.0.2=py37_1
    - pycosat==0.6.3=py37h7b6447c_0
    - pycparser==2.19=py37_0
    - pyopenssl==19.1.0=py37_0
    - pysocks==1.7.1=py37_0
    - python==3.7.6=h0371630_2
    - readline==7.0=h7b6447c_5
    - requests==2.22.0=py37_1
    - ruamel_yaml==0.15.87=py

In [3]:
! conda env update -n base -f reinvent_shared.yml
! pip install https://github.com/josecarlosgomezt/nonconformist/archive/master.zip
! pip install lightgbm==2.2.3
! pip install scikit-learn==0.22
# ! pip install pandas==0.25.3

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

## Code Execution Normal REINVENT

In [ ]:
# Import packages
import os
import json
import numpy as np

# Input =====================================================
templates_folder = 'data/examples/templates'
name_model = 'Training/base_model/base_model.ckpt'
dataset = 'data/CHEMBL_FULL_DATASET.txt'
tuned_model = 'Training/transfer_learning/tuned_model.ckpt'
dataset_active = 'data/active_chembl_27.txt'
batch_size = 128
epochs = 120
output_smiles = 'Results/Outputs/REINVENT_tuned_molecules.txt'
num_smiles = 5100
# ===========================================================


def find_latest_checkpoint(name_model):
    model_name = name_model.split('/')[-1]
    path = name_model.replace('/' + model_name, '')
    files = os.listdir(path)
    values = []
    model_name += '.'
    for item in files:
        if model_name in item:
            values.append(int(item.split('.')[-1]))
        elif 'ipynb' in item:
            values.append(-1)
        else:
            values.append(0)
    return os.path.join(path, files[np.argmax(np.asarray(values))])

# find_latest_checkpoint(tuned_model)

In [ ]:
# # CREATE MODEL
# # write the configuration file to the disc
# configuration_JSON_path = os.path.join(templates_folder, "create_model.json")
# with open(configuration_JSON_path, 'r') as f:
#     data = json.load(f)
#     data["parameters"]["output_model_path"] = name_model
#     data["parameters"]["input_smiles_path"] = dataset
#     print(data)

# with open(configuration_JSON_path, 'w') as f:
#     json.dump(data, f, indent=4, sort_keys=True)

# ! python input.py data/examples/templates/create_model.json

In [ ]:
# TRAIN MODEL
configuration_JSON_path = os.path.join(templates_folder, "transfer_learning.json")
with open(configuration_JSON_path, 'r') as f:
    data = json.load(f)
    data["parameters"]["input_model_path"] = find_latest_checkpoint(tuned_model)
    data["parameters"]["output_model_path"] = tuned_model
    data["parameters"]["input_smiles_path"] = dataset_active
    data["parameters"]["batch_size"] = batch_size
    data["parameters"]["num_epochs"] = epochs
    data["parameters"]["save_every_n_epochs"] = 10
    print(data)

with open(configuration_JSON_path, 'w') as f:
    json.dump(data, f, indent=4, sort_keys=True)

! python input.py data/examples/templates/transfer_learning.json

{'logging': {'job_id': 'relevant for remote logging', 'job_name': 'transfer learning', 'logging_path': 'Results/logging/progress.log', 'recipient': 'local', 'sender': 'this is relevant for remote job submission scenario', 'use_weights': 0}, 'parameters': {'adaptive_lr_config': {'average_steps': 4, 'gamma': 0.8, 'min': 1e-05, 'mode': 'constant', 'patience': 8, 'restart_times': 0, 'restart_value': 1e-05, 'sample_size': 100, 'start': 0.0005, 'step': 1, 'threshold': 0.0001}, 'batch_size': 128, 'clip_gradient_norm': 1.0, 'collect_stats_frequency': 1, 'input_model_path': 'Training/transfer_learning/tuned_model.ckpt.130', 'input_smiles_path': 'data/active_chembl_27.txt', 'num_epochs': 120, 'output_model_path': 'Training/transfer_learning/tuned_model.ckpt', 'randomize': True, 'save_every_n_epochs': 10, 'shuffle_each_epoch': 1, 'standardize': True, 'starting_epoch': 1}, 'run_type': 'transfer_learning', 'version': '2.0'}
/usr/local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/d

In [ ]:
# GENERATE MODLECULES
configuration_JSON_path = os.path.join(templates_folder, "sampling.json")
with open(configuration_JSON_path, 'r') as f:
    data = json.load(f)
    data["parameters"]["model_path"] = find_latest_checkpoint(tuned_model)
    data["parameters"]["output_smiles_path"] = output_smiles
    data["parameters"]["num_smiles"] = num_smiles
    data["parameters"]["batch_size"] = batch_size
    print(data)

with open(configuration_JSON_path, 'w') as f:
    json.dump(data, f, indent=4, sort_keys=True)
    
! python input.py data/examples/templates/sampling.json

{'logging': {'job_name': 'sampling', 'logging_path': 'Results/logging/progress.log', 'recipient': 'local', 'sender': 'this is relevant for remote job submission scenario'}, 'parameters': {'batch_size': 128, 'model_path': 'Training/transfer_learning/tuned_model.ckpt.250', 'num_smiles': 5100, 'output_smiles_path': 'Results/Outputs/REINVENT_tuned_molecules.txt', 'with_likelihood': False}, 'run_type': 'sampling', 'version': '2.0'}
/usr/local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.d

## Code Execution RL REINVENT

In [5]:
# Import packages
import os
import json
import numpy as np

# Input =====================================================
templates_folder = 'data/examples/templates'
name_model = 'Training/base_model/base_model.ckpt'
rl_model = 'Training/RL/base_model.ckpt'
output_smiles = 'Results/Outputs/REINVENT_RL_sa_qed_gsk.txt'
result_dir = "Training/RL/results_sa_qed_gsk" 
batch_size = 128
num_smiles = 5100
# ===========================================================


def find_latest_checkpoint(name_model):
    model_name = name_model.split('/')[-1]
    path = name_model.replace('/' + model_name, '')
    files = os.listdir(path)
    values = []
    model_name += '.'
    for item in files:
        if model_name in item:
            values.append(int(item.split('.')[-1]))
        elif 'ipynb' in item:
            values.append(-1)
        else:
            values.append(0)
    return os.path.join(path, files[np.argmax(np.asarray(values))])

# find_latest_checkpoint(tuned_model)

In [ ]:
# TRAIN MODEL
configuration_JSON_path = os.path.join(templates_folder, "reinforcement_learning.json")
with open(configuration_JSON_path, 'r') as f:
    data = json.load(f)
    data["parameters"]['reinforcement_learning']["prior"] = find_latest_checkpoint(name_model)
    data["parameters"]['reinforcement_learning']["agent"] = find_latest_checkpoint(rl_model)
    data["logging"]["resultdir"] = result_dir
    data["parameters"]['reinforcement_learning']["batch_size"] = batch_size
    data["parameters"]['reinforcement_learning']["n_steps"] = 2000
    # data["parameters"]["save_every_n_epochs"] = 10
    print(data)

with open(configuration_JSON_path, 'w') as f:
    json.dump(data, f, indent=4, sort_keys=True)

! python input.py data/examples/templates/reinforcement_learning.json

Output streaming troncato alle ultime 5000 righe.
07:41:33: local_reinforcement_logger.timestep_report +40: INFO     
 Step 568   Fraction valid SMILES: 100.0   Score: 0.3977   Time elapsed: 654   Time left: 1645.9
  Agent     Prior     Target     Score     SMILES
-20.96    -46.09     24.39      0.55      c1(-c2c(-c3ccccn3)cc(NS(=O)(CCC)=O)cc2)c[nH]c(-c2cnccc2)c1
-14.06    -34.00     44.79      0.62      c1(-c2cncnc2)cc(NS(c2cccnc2)(=O)=O)ccc1-c1ccco1
-17.50    -42.50     34.52      0.60      c1c(-c2cc(-c3cccc(NS(=O)(=O)CC4CC4)c3)ccc2NC(=O)C)cncc1C#N
-19.60    -42.51     34.89      0.60      c1(-c2cc(NS(=O)(=O)N3CCC3)ccc2-c2ccn[nH]2)cc(N2CCOCC2)cnc1
-19.88    -42.32     35.88      0.61      c1c(Oc2c(-c3cccc(N)c3)cc(NS(=O)(=O)C3CC3)cc2)c(C#N)ccn1
-22.80    -46.57     30.70      0.60      c1(-c2cc3c(cc2)OC(C2CNC2=O)C3)cc(N2CCNC2=O)ccc1C#N
-23.80    -39.69     35.26      0.59      c1(=O)c2ccc(NS(=O)(c3cccnc3)=O)cc2[nH]cc1-c1cccc(N2CCCC2)c1
-21.50    -37.77     38.05      0.59      S1(=O)(

In [ ]:
# GENERATE MODLECULES
configuration_JSON_path = os.path.join(templates_folder, "sampling.json")
with open(configuration_JSON_path, 'r') as f:
    data = json.load(f)
    data["parameters"]["model_path"] = "Training/RL/results_sa_qed_gsk/Agent.ckpt"
    data["parameters"]["output_smiles_path"] = output_smiles
    data["parameters"]["num_smiles"] = num_smiles
    data["parameters"]["batch_size"] = batch_size
    print(data)

with open(configuration_JSON_path, 'w') as f:
    json.dump(data, f, indent=4, sort_keys=True)
    
! python input.py data/examples/templates/sampling.json

{'logging': {'job_name': 'sampling', 'logging_path': 'Results/logging/progress.log', 'recipient': 'local', 'sender': 'this is relevant for remote job submission scenario'}, 'parameters': {'batch_size': 128, 'model_path': 'Training/RL/results_sa_qed_gsk/Agent.ckpt', 'num_smiles': 5100, 'output_smiles_path': 'Results/Outputs/REINVENT_RL_sa_qed_gsk.txt', 'with_likelihood': False}, 'run_type': 'sampling', 'version': '2.0'}
/usr/local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("